Let's use what we did in the last notebook to grab some headlines and use them to generate headlines

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from urllib.request import urlopen, Request
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

2024-07-23 16:57:24.194267: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-23 16:57:24.194343: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-23 16:57:24.194981: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-23 16:57:24.200405: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Obtain the dataset

In [2]:
n = 3 #the # of article headlines displayed per ticker
tickers = ['AAPL', 'NVDA', 'TSLA', 'MSFT']

In [3]:
# Get Data
headers = {
        "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'
    }
finwiz_url = 'https://finviz.com/quote.ashx?t='
news_tables = {}

for ticker in tickers:
    url = finwiz_url + ticker
    req = Request(url=url,headers=headers) 
    resp = urlopen(req)    
    html = BeautifulSoup(resp, features="lxml")
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table

In [4]:
try:
    for ticker in tickers:
        df = news_tables[ticker]
        df_tr = df.findAll('tr')
    
        print ('\n')
        print ('Recent News Headlines for {}: '.format(ticker))
        
        for i, table_row in enumerate(df_tr):
            a_text = table_row.a.text
            td_text = table_row.td.text
            td_text = td_text.strip()
            print(a_text,'(',td_text,')')
            if i == n-1:
                break
except KeyError:
    pass



Recent News Headlines for AAPL: 
Is NVIDIA Corp (NASDAQ:NVDA) the Best AI Stock to Buy Based on New AI ETF? ( Today 05:51PM )
What Verizon's Low Smartphone Upgrades Mean For Apple ( 05:16PM )
Magnificent Seven Stocks: Nvidia Stock Falls, Tesla Slides Ahead Of Earnings; Google Earnings Next ( 03:22PM )


Recent News Headlines for NVDA: 
Is Amazon.com Inc (NASDAQ:AMZN) the Best AI Stock to Buy Now? ( Today 06:01PM )
Is Microsoft Corp (NASDAQ:MSFT) the Best AI Stock to Buy Based on New AI ETF? ( 05:55PM )
Is Meta Platforms Inc. (NASDAQ:META) the Best AI Stock to Buy Based on New AI ETF? ( 05:53PM )


Recent News Headlines for TSLA: 
Tesla Earnings Fall 43% In 'Difficult Operating Environment'; Elon Musk Touts Autonomy (Live Coverage) ( Today 06:48PM )
To ignore robotaxis is a huge mistake with Tesla's stock, says ARK Invest's Tasha Keeney ( 06:19PM )
Teslas Mexico Production Plans Depend on U.S. Election, Musk Says ( 06:12PM )


Recent News Headlines for MSFT: 
Is Microsoft Corp (NASDAQ

In [5]:
# Iterate through the news, scraping the text
parsed_news = []
for file_name, news_table in news_tables.items():  # use the ticker as the file name
    for x in news_table.findAll('tr'):
        if x.a == None:
            continue
        else:
            text = x.a.text 
            date_scrape = x.td.text.split()  # text

            if len(date_scrape) == 1:  # time
                time = date_scrape[0]
                
            else:
                date = date_scrape[0]  # date and time
                time = date_scrape[1]

            ticker = file_name.split('_')[0]  # Ticker Symbol
            
            parsed_news.append([ticker, date, time, text])
        

In [6]:
parsed_news

[['AAPL',
  'Today',
  '05:51PM',
  'Is NVIDIA Corp (NASDAQ:NVDA) the Best AI Stock to Buy Based on New AI ETF?'],
 ['AAPL',
  'Today',
  '05:16PM',
  "What Verizon's Low Smartphone Upgrades Mean For Apple"],
 ['AAPL',
  'Today',
  '03:22PM',
  'Magnificent Seven Stocks: Nvidia Stock Falls, Tesla Slides Ahead Of Earnings; Google Earnings Next'],
 ['AAPL', 'Today', '02:36PM', "Why It's Time to Buy the Dip in Tech Stocks"],
 ['AAPL',
  'Today',
  '12:53PM',
  'Apple likely to release foldable iPhone as early as 2026, the Information reports'],
 ['AAPL', 'Today', '12:32PM', 'Spotify Keeps the Profit Tune Humming'],
 ['AAPL',
  'Today',
  '12:08PM',
  'Meta CEO Zuckerberg calls on industry to adopt open-source AI, debuts high-powered Llama AI model'],
 ['AAPL',
  'Today',
  '11:28AM',
  'Apple wants to stop spending so much money making TV shows'],
 ['AAPL',
  'Today',
  '11:21AM',
  'Behind the Historic Small Cap Breakout, and Investing Lessons for Millennials'],
 ['AAPL',
  'Today',
  '0

In [7]:
type(parsed_news)

list

In [8]:
headlines = []
for l in parsed_news:
    headlines.append(l[3])


In [9]:
headlines

['Is NVIDIA Corp (NASDAQ:NVDA) the Best AI Stock to Buy Based on New AI ETF?',
 "What Verizon's Low Smartphone Upgrades Mean For Apple",
 'Magnificent Seven Stocks: Nvidia Stock Falls, Tesla Slides Ahead Of Earnings; Google Earnings Next',
 "Why It's Time to Buy the Dip in Tech Stocks",
 'Apple likely to release foldable iPhone as early as 2026, the Information reports',
 'Spotify Keeps the Profit Tune Humming',
 'Meta CEO Zuckerberg calls on industry to adopt open-source AI, debuts high-powered Llama AI model',
 'Apple wants to stop spending so much money making TV shows',
 'Behind the Historic Small Cap Breakout, and Investing Lessons for Millennials',
 "Meta's reality check: Inside the $45 billion cash burn at Reality Labs",
 'Is Apple Inc (NASDAQ:AAPL) the Best Trump Trade According to Jim Cramer?',
 'Could Earnings Stop the Small-Cap Stocks Rotation in Its Tracks?',
 'Marjorie Taylor Greene Loves AI Stocks. Here Are 5 She Bought in the First Half of 2024.',
 '3 Warren Buffett Stoc

# Tokenize the data

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Tokenize the words in our headlines
tokenizer = Tokenizer()
tokenizer.fit_on_texts(headlines)
total_words = len(tokenizer.word_index) + 1
print('Total words: ', total_words)

Total words:  1401


Let's look at the tokenization

In [11]:
subset_dict = {key: value for key, value in tokenizer.word_index.items() \
               if key in ['an','apple','a','day']}
print(subset_dict)

{'a': 12, 'apple': 17, 'an': 122, 'day': 443}


In [12]:
tokenizer.texts_to_sequences(['an','apple','a','day'])

[[122], [17], [12], [443]]

In [13]:
# Convert data to sequence of tokens 
input_seqs = []
for hl in headlines:
    # Convert each headline into a sequence of tokens
    tokens_list = tokenizer.texts_to_sequences([hl])[0]
    
    # each headline is converted into a series of sequences
    for i in range(1, len(tokens_list)):
        partial_seq = tokens_list[:i+1]
        input_seqs.append(partial_seq)

print(tokenizer.sequences_to_texts(input_seqs[:10]))
input_seqs[:10]

['is nvidia', 'is nvidia corp', 'is nvidia corp nasdaq', 'is nvidia corp nasdaq nvda', 'is nvidia corp nasdaq nvda the', 'is nvidia corp nasdaq nvda the best', 'is nvidia corp nasdaq nvda the best ai', 'is nvidia corp nasdaq nvda the best ai stock', 'is nvidia corp nasdaq nvda the best ai stock to', 'is nvidia corp nasdaq nvda the best ai stock to buy']


[[4, 7],
 [4, 7, 100],
 [4, 7, 100, 44],
 [4, 7, 100, 44, 77],
 [4, 7, 100, 44, 77, 3],
 [4, 7, 100, 44, 77, 3, 34],
 [4, 7, 100, 44, 77, 3, 34, 5],
 [4, 7, 100, 44, 77, 3, 34, 5, 2],
 [4, 7, 100, 44, 77, 3, 34, 5, 2, 1],
 [4, 7, 100, 44, 77, 3, 34, 5, 2, 1, 16]]

## Add Padding
We need to add padding in order to make all sequences the same length

In [14]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Determine max sequence length
max_sequence_len = max([len(x) for x in input_seqs])

# Pad all sequences with zeros at the beginning to make them all max length
input_seqs = np.array(pad_sequences(input_seqs, maxlen=max_sequence_len, padding='pre'))
input_seqs[6]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   4,   7, 100,  44,  77,   3,  34,   5], dtype=int32)

Obviously we have some longer headlines...

# Create Predictors

In [15]:
# Everything except the last word is used as a predictor for the last words (targets)
# this is essentially X_train
predictors = input_seqs[:,:-1]

# Create Targets

In [16]:
from tensorflow.keras import utils


targets = input_seqs[:,-1] # targets are the last word in the sentence

print(targets)
print(total_words)

# make the targets categorical (a sparse array of one-hot-encoded lists)
targets = utils.to_categorical(targets, num_classes = total_words)

len(targets[0])
len(targets)

[   7  100   44 ... 1399  191 1400]
1401


4361

# Model

In [17]:
input_len = len(input_seqs[0])
input_len

35

In [18]:
max_sequence_len

35

In [19]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential

# Input is max sequence length - 1, as we've removed the last word for the label
#input_len = len(input_seqs[0])-1
input_len = max_sequence_len -1

# Using the Sequential API
model = Sequential()

# Embedding layer
model.add(Embedding(total_words, 10, input_length=input_len)) # input dim, output dim, input length

# Add LSTM layer with 100 units
model.add(LSTM(100)) # very basic LSTM
model.add(Dropout(0.1))

# Add output layer - softmax at the end to set the logits to probabiilty
model.add(Dense(total_words, activation='softmax'))





2024-07-23 16:57:27.882940: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-23 16:57:27.913092: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-23 16:57:27.913248: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-23 16:57:27.917754: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-23 16:57:27.917886: I external/local_xla/xla/stream_executor

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 34, 10)            14010     
                                                                 
 lstm (LSTM)                 (None, 100)               44400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1401)              141501    
                                                                 
Total params: 199911 (780.90 KB)
Trainable params: 199911 (780.90 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Compile the model

In [21]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

Train the model

In [22]:
model.fit(predictors, 
        targets, 
        epochs=150, 
        verbose=1)

Epoch 1/150


2024-07-23 16:57:34.016806: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-07-23 16:57:34.731540: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f989112eed0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-23 16:57:34.731584: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090 Ti, Compute Capability 8.6
2024-07-23 16:57:34.736808: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1721775454.836575 3218974 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


137/137 [==============================] - 17s 102ms/step - loss: 6.7205
Epoch 2/150
137/137 [==============================] - 4s 30ms/step - loss: 6.3549
Epoch 3/150
137/137 [==============================] - 3s 21ms/step - loss: 6.2945
Epoch 4/150
137/137 [==============================] - 3s 19ms/step - loss: 6.2408
Epoch 5/150
137/137 [==============================] - 2s 18ms/step - loss: 6.1862
Epoch 6/150
137/137 [==============================] - 2s 18ms/step - loss: 6.1328
Epoch 7/150
137/137 [==============================] - 2s 16ms/step - loss: 6.0697
Epoch 8/150
137/137 [==============================] - 2s 17ms/step - loss: 5.9984
Epoch 9/150
137/137 [==============================] - 2s 13ms/step - loss: 5.9092
Epoch 10/150
137/137 [==============================] - 2s 15ms/step - loss: 5.8004
Epoch 11/150
137/137 [==============================] - 2s 16ms/step - loss: 5.6879
Epoch 12/150
137/137 [==============================] - 2s 14ms/step - loss: 5.5627
Epoch 13/15

# Make Predictions

In [23]:
tokens = tokenizer.texts_to_sequences(["Apple recently announced"])[0]

In [24]:
tokens

[17]

In [25]:
tokens = pad_sequences([tokens], maxlen=max_sequence_len-1,padding='pre') # pad the beginning
prediction = np.argmax(model.predict(tokens), axis=-1)

#tokenizer.sequences_to_texts([prediction])

1/1 [==============================] - 0s 327ms/step


In [26]:
tokenizer.sequences_to_texts([prediction])

['aapl']

In [27]:
# make a function for the above
def generate_next_token(text):
    tokens = tokenizer.texts_to_sequences([text])[0]
    tokens = pad_sequences([tokens], maxlen=max_sequence_len-1,padding='pre') # pad the beginning
    prediction = np.argmax(model.predict(tokens), axis=-1)
    return prediction

In [28]:
def generate_sentence (text, num_words):

    for _ in range (num_words):
        pred = generate_next_token(text)
        next = tokenizer.sequences_to_texts([pred])[0]
        text += " " + next
    return text

In [29]:
print(generate_sentence("Apple recently announced", num_words=10))
print(generate_sentence("NVIDIA", num_words=15))
print(generate_sentence("What happened to", num_words=15))
print(generate_sentence("CrowdStrike", num_words=15))

1/1 [==============================] - 0s 21ms/step
Apple recently announced aapl re emerged as a top performer in q2 stock
1/1 [==============================] - 0s 21ms/step
NVIDIA stock rises after big tech rebound this will drive its next move ai stock to
1/1 [==============================] - 0s 20ms/step
What happened to need to know ahead of microsoft's earnings new china chip for ai ai etf for
1/1 [==============================] - 0s 28ms/step
CrowdStrike shares trim heavy losses as microsoft announces resolution of global tech outage 'this is clearly
